In [ ]:
# 디자인 시험 합격여부 관련 문제 
# id 기준 140번 이전까지 학습데이터, 이후는 테스트데이터로 분리
# (본데이터 아이디순서대로 정렬되어있음)

# 1. 주어진 데이터의 모든 변수를 활용해 design을 예측하는 다중회귀분석을 시행한 후
# (패널티는 부과되지 않으며, 모델은 절편항을 포함한다)
# 유의하지 않은 설명변수 개수를 구하라(정수)

import pandas as pd
from statsmodels.formula.api import ols

df = pd.read_csv('경로')

# 데이터 분리
train = df[df.id <= 140] #df.iloc[:140] 가능
test - df[df.id > 140]

# 데이터 학습 및 결과
model = ols("design ~ tenure + f2 + f3 + f4 + f5", data=train).fit()
print(model.summary())
print(model.pvalues < 0.05) 

# intercept True
# tenure False
# f2 True
# f3 False
# f4 True
# f5 False

# 유의하지 않은 설명변수 = 2개 (설명변수(=독립변수)에는 절편항 포함 X)

# 이삭님 풀이
import pandas as pd
import statsmodels.api as sm
df = pd.read_csv('경로')

train = df.iloc[:140]
test - df.iloc[140:]

y = train['design']
X = train[['변수1', '변수2' ,'변수3', '변수4']]

X = sm.add_constant(X)
model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

#여기까지 하면 다중회귀 결과가 나오고 
#p > |t| 값이 0.05보다 작은 설명변수의 개수를 구하면 2개 
#(절편은 설명변수가 아니기 때문에 개수 포함 X)

#2번. # 2. f2, f4 변수를 활용하여 design을 예측하는 다중분석회귀를 시행한 후, 
# !학습용데이터!의 예측값과 실제값의 피어슨 상관계수를 구하시오.

model = ols("design ~ f2 + f4", data=train).fit()
y_pred = model.predict(train)
y_real = train['design']

pd.DataFrame({'y_real':y_real, 'y_pred' : y_pred}).corr().iloc[0:1].values
# 피어슨이 아니었다면? method = 'kendal', 'spearman' ...
# 답 0.397

# 이삭님 풀이
X2 = train[['변수1', '변수2']] 
X2 = sm.add_constant(X2)

model2 = sm.OLS(y, X2)
result2 = model2.fit()
print(result2.summary())

pred_y = result2.predict(X2)

from scipy.stats import pearsonr
rho, pvalue = pearsonr(y, pred_y)
print(round(rho, 3))

# 3. 2번 모델로 적합한 모델로 테스트데이터의 RMSE를 구하시오
from sklearn.metrics import mean_squared_error

y_pred = model.predict(test)
print(round((mean_sqaured_error(test['design'], y_pred) ** 0.5), 3)) # 8.098

#작업형 3-2 (로지스틱회귀분석)
# 1. 주어진 데이터를 활용해 이탈을 예측하는 로지스틱 회귀를 시행한 후
# col1의 pvalue를 구하시오

import pandas as pd
from statsmodels.formula.api import logit

model = logit("churn ~ col1 + col2 + Phone_Service + Tech_Insurance", data=df).fit()
print(model.summary())
print(round(model.pvalues['col1'], 3)) # 답 0.047

# 이삭님 풀이
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv('경로')

y = df['타겟변수']
X = df.drop('타겟변수', axis=1)
X = sm.add_constant(X)

model = sm.GLM(y, X, family=sm.families.Binomial())
result = model.fit()
print(result.summary())

#여기까지 하면 로지스틱 회귀분석 결과가 나옴 
#거기서 1번 문제의 컬럼에 대한 유의확률을 소수점 3자리까지 반올림 하면 0.047

# 2. 폰서비스를 받지 않은 고객이 이탈할 확률 대비 
# 폰서비스를 받은 고객이 이탈할 확률에 대한 오즈비를 구하시오.
print(round(np.exp(model.params['Phone_Service']), 3)) # 답 0.655 

# 이삭님 풀이
#log(odds) = beta이므로 odds = e^(beta)가 됨. 
#여기서 다른 변수의 영향은 무시하고 2번 문제에서 요구한 변수가 0에서 1이 됐을 때의 오즈비이므로 
#e^(변수2의 coef)값을 계산하면 됨. (왜냐하면 변수가 0일때의 odds는 e^0이 돼서 분모가 1이라 무시해도 되기 때문) 
#일단 result.params로 각 변수들의 회귀계수를 파악하고 result.params.loc['변수명']으로 회귀계수를 가져온 뒤에 
#np.exp(result.params.loc['변수명'])을 계산하면 오즈비 계산 가능. 
#round 함수로 소수점 3자리까지 나타내면 답은 0.655

# 3. 데이터로 예측한 후 이탈할 확률이 0.3 이상인 고객들의 수를 구하시오.
pred = result.predict(df)
print(sum(pred > 0.3))  # 225

# 이삭님 풀이
sum(result.predict(X) > 0.3) 

#로지스틱 회귀분석으로 만든 모델로 실제로 X데이터를 넣어 고객이 이탈 할 확률값을 출력한 다음 
#그 확률값이 0.3보다 큰 고객의 수를 계산하는 문제였는데 
#그냥 resuslt.predict(X)를 넣으면 이탈 할 확률값이 나오고 result.predict(X) > 0.3을 하면 
#boolean타입으로 큰지 작은지 True False로 나오니 
#그 결과를 그냥 sum(result.predict(X) > 0.3) 해주면 225가 나옴